In [ ]:
!pip install transformers datasets peft torch

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
import torch

In [ ]:
dataset = load_dataset("huggingartists/kendrick-lamar")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.22k [00:00<?, ?B/s]

kendrick-lamar.py:   0%|          | 0.00/4.08k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/1.42M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/861 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 861
    })
})

In [ ]:
dataset["train"][0]

{'text': 'Nobody pray for me\nIt been that day for me\nWay \nAyy, I remember syrup sandwiches and crime allowances\nFinesse a nigga with some counterfeits, but now Im countin’ this\nParmesan where my accountant lives, in fact Im downin this\nD’USSÉ with my boo bae tastes like Kool-Aid for the analysts\nGirl, I can buy your ass the world with my paystub\nOoh, that pussy good, wont you sit it on my taste bloods?\nI get way too petty once you let me do the extras\nPull up on your block, then break it down: we playin Tetris\nAM to the PM, PM to the AM, funk\nPiss out your per diem, you just gotta hate em, funk\nIf I quit your BM, I still ride Mercedes, funk\nIf I quit this season, I still be the greatest, funk\nMy left stroke just went viral\nRight stroke put lil baby in a spiral\nSoprano C, we like to keep it on a high note\nIts levels to it, you and I know\nBitch, be humble \nSit down \nBe humble \nSit down \nBe humble \nBitch, sit down \nBe humble \nSit down \nBe humble \nSit down \nBe 

In [ ]:
tokeniser = AutoTokenizer.from_pretrained("gpt2", use_fast=True)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
tokeniser.pad_token = tokeniser.eos_token

In [ ]:
def tokenise_function(examples):
  return tokeniser(examples['text'])

In [ ]:
tokenised_dataset = dataset.map(tokenise_function, batched=True, num_proc=4, remove_columns=['text'])

Map (num_proc=4):   0%|          | 0/861 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1053 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1044 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1061 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1202 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
tokenised_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 861
    })
})

In [ ]:
tokenised_dataset['train']['input_ids'][0]

[24795,
 12472,
 329,
 502,
 198,
 1026,
 587,
 326,
 1110,
 329,
 502,
 198,
 25309,
 220,
 198,
 32,
 22556,
 11,
 314,
 3505,
 24208,
 29777,
 290,
 4065,
 42290,
 198,
 37,
 1127,
 325,
 257,
 44873,
 4908,
 351,
 617,
 29465,
 896,
 11,
 475,
 783,
 1846,
 954,
 259,
 447,
 247,
 428,
 198,
 47,
 1670,
 42890,
 810,
 616,
 42627,
 3160,
 11,
 287,
 1109,
 1846,
 866,
 259,
 428,
 198,
 35,
 447,
 247,
 32835,
 38351,
 351,
 616,
 22513,
 275,
 3609,
 18221,
 588,
 509,
 970,
 12,
 44245,
 329,
 262,
 13440,
 198,
 24151,
 11,
 314,
 460,
 2822,
 534,
 840,
 262,
 995,
 351,
 616,
 1414,
 301,
 549,
 198,
 46,
 1219,
 11,
 326,
 18185,
 922,
 11,
 28329,
 345,
 1650,
 340,
 319,
 616,
 6938,
 2910,
 82,
 30,
 198,
 40,
 651,
 835,
 1165,
 25229,
 1752,
 345,
 1309,
 502,
 466,
 262,
 33849,
 198,
 42940,
 510,
 319,
 534,
 2512,
 11,
 788,
 2270,
 340,
 866,
 25,
 356,
 711,
 259,
 27351,
 2442,
 198,
 2390,
 284,
 262,
 3122,
 11,
 3122,
 284,
 262,
 3001,
 11,
 46212,
 198,
 47,


In [ ]:
block_size = 128

In [ ]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # assuming each key have list (their values) of same length, get the length of the lists
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # Floor divide to get perfect divisor of block_size and multiply back to get total length (drop remainder)
    # We could add padding if the model supported it instead of dropping.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {}
    for k,t in concatenated_examples.items():
        chunks = [t[i: i + block_size] for i in range(0, total_length, block_size)]

        # check if last chunk needs padding
        if len(chunks[-1]) < block_size:
            num_padding = block_size - len(chunks[-1])
            # denote padding_value = 0, and add padding to make the chunk size = block_size
            chunks[-1] = chunks[-1] + [0] * num_padding

        result[k] = chunks
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_datasets = tokenised_dataset.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Map (num_proc=4):   0%|          | 0/861 [00:00<?, ? examples/s]

In [ ]:
lm_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 5169
    })
})

In [ ]:
tokeniser.decode(lm_datasets["train"][1]["input_ids"])


' up on your block, then break it down: we playin Tetris\nAM to the PM, PM to the AM, funk\nPiss out your per diem, you just gotta hate em, funk\nIf I quit your BM, I still ride Mercedes, funk\nIf I quit this season, I still be the greatest, funk\nMy left stroke just went viral\nRight stroke put lil baby in a spiral\nSoprano C, we like to keep it on a high note\nIts levels to it, you and I know\nBitch, be humble \nSit down \nBe humble \nSit'

In [ ]:
model = AutoModelForCausalLM.from_pretrained("gpt2")

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
training_args = TrainingArguments(
    f"gpt2-finetuned-kendrick_lamar",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=3e-4,
    weight_decay=1e-4,
    report_to="none"
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
)

In [ ]:
trainer.train()


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


TrainOutput(global_step=323, training_loss=3.831612507256192, metrics={'train_runtime': 206.3705, 'train_samples_per_second': 25.047, 'train_steps_per_second': 1.565, 'total_flos': 337589305344000.0, 'train_loss': 3.831612507256192, 'epoch': 0.9992266047950503})

In [ ]:
model.save_pretrained("./models")

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
model2 = AutoModelForCausalLM.from_pretrained('./models').to(device)

In [ ]:
torch.manual_seed(42)


In [ ]:
input_ids = tokeniser.encode("Write me a Kendrick Lamar song \n", return_tensors="pt").to(device)


In [ ]:
input_ids

tensor([[16594,   502,   257, 38643, 26907,  3496,   220,   198]],
       device='cuda:0')

In [ ]:
sample_output = model.generate(
    input_ids,
    do_sample=True,
    max_length=500,
    top_k=50,
    top_p=0.95,
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
sample_output.shape

torch.Size([1, 500])

In [ ]:
generated_text = tokeniser.decode(sample_output[0], skip_special_tokens=True)
print(generated_text)

Write me a Kendrick Lamar song 
What I dont love about her, I love her 
She needs me, she needs me, she need me, she need me, she need me, she want me
I love her, I love her, I love her
I love her, I love her, I love her
So we go and we dance to God
Now let me put them two on the CD
Now let me put them two on the CD
Now let me put these two on the CD
I love her, I love her, I love her
And let us play that part
Let me put these two on the CD
Ohhh, ayy, ayy
And let the CD get the same
Ayy, let the CD get the same
Ayy, ayy, Ayy, ayy
Uh, I know youre gonna keep me on this track, I might get lost, I might get lost 
Okay, I know I got your number, I know I might get lost
I just gotta get this right in the booth, or somethin aint the answer
But I got the stamina to get me to stop
Cause Im from the grindin, Ive just got to run and get on the shit, and get it right 
Hey, ayy, ayy, ayy
Just a nigga wanna get something, nigga 
I dont even care who you gonna be on that, nigga
Uh, I just want my sh

In [ ]:
with open("gpt2_text_output.txt", "w") as f:
    f.write(generated_text)